In [1]:
import os
import yaml
import pandas as pd

# 列幅を広げる（制限をなくす場合は -1 を指定）
pd.options.display.max_colwidth = None

# 設定ファイルの読み込み
with open('config.yaml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

primary_folders = config['primary_folders']
secondary_folders = config['secondary_folders']
ignore_extensions = config['ignore_extensions']

In [2]:
# 患者候補フォルダのリストを格納するリスト
patient_candidates = []
exception_entries = []

# primary_folderとsecondary_folderの絶対パスを記録
primary_paths = {os.path.abspath(folder) for folder in primary_folders}
secondary_paths = {os.path.abspath(folder) for folder in secondary_folders}

# secondary_folder直下の患者候補フォルダをリストアップ
patient_paths = set()
for secondary_folder in secondary_folders:
    abs_secondary_folder = os.path.abspath(secondary_folder)
    if os.path.exists(abs_secondary_folder):
        for entry in os.listdir(abs_secondary_folder):
            entry_path = os.path.join(abs_secondary_folder, entry)
            if os.path.isdir(entry_path):
                patient_candidates.append({
                    'patient_folder_name': entry,
                    'patient_folder_path': os.path.abspath(entry_path),
                    'secondary_folder': abs_secondary_folder
                })
                # 患者候補フォルダとその配下を追加
                for root, dirs, files in os.walk(entry_path):
                    patient_paths.add(os.path.abspath(root))

# 患者候補フォルダ一覧をDataFrameに変換して保存
df_patients = pd.DataFrame(patient_candidates)
df_patients.to_parquet('patients_candidates.parquet', index=False)
print("患者候補フォルダの一覧をpatients_candidates.parquetとして出力しました。")

患者候補フォルダの一覧をpatients_candidates.parquetとして出力しました。


In [3]:
import gc

# primary_folderの処理
for primary_folder in primary_folders:
    abs_primary_folder = os.path.abspath(primary_folder)

    for dirpath, dirnames, filenames in os.walk(abs_primary_folder):
        abs_dirpath = os.path.abspath(dirpath)

        # primary_folder自身だけを除外（完全一致の条件を使用）
        if abs_dirpath in abs_primary_folder:
            continue

        # secondary_folder自身だけを除外（完全一致の条件を使用）
        if abs_dirpath in {os.path.abspath(secondary_folder) for secondary_folder in secondary_folders}:
            continue

        # 患者候補フォルダおよびその配下を除外（親子関係を判定）
        if any(abs_dirpath.startswith(patient_path) for patient_path in patient_paths):
            continue

        # 無視する拡張子のファイルを除外
        filenames = [f for f in filenames if not any(f.lower().endswith(ext.lower()) for ext in ignore_extensions)]

        # 残ったフォルダやファイルを例外リストに追加
        for dirname in dirnames:
            entry_path = os.path.join(dirpath, dirname)
            exception_entries.append({
                'type': 'folder',
                'path': os.path.abspath(entry_path)
            })
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            exception_entries.append({
                'type': 'file',
                'path': os.path.abspath(file_path)
            })

# 変数の削除（jupyter用）
del df_exceptions
gc.collect()

# 例外フォルダ・ファイル一覧をDataFrameに変換して保存
df_exceptions = pd.DataFrame(exception_entries)
df_exceptions.to_parquet('exceptions.parquet', index=False)
print("例外として検出されたファイル・フォルダをexceptions.parquetとして出力しました。")

df_exceptions

NameError: name 'df_exceptions' is not defined